In [9]:
import sys

sys.path.append('..')
import numpy as np
import chess
from stockfish import Stockfish
from lczero.backends import Backend, Weights, GameState

In [10]:
stockfish = Stockfish()
maia = Backend(Weights('../data/maia/1600.pb.gz'))

Creating backend [eigen]...
Using Eigen version 3.4.0
Eigen max batch size is 256.


In [11]:
def stockfish_probs(fen):
  b = chess.Board(fen)
  evals = []
  for m in b.legal_moves:
    b.push(m)
    stockfish.set_fen_position(b.fen())
    evaluation = stockfish.get_evaluation()
    if evaluation['type'] == 'cp':
      evals.append(evaluation['value'] / 100)
    else:
      evals.append(1 / ((2 * evaluation['value']) + 1) * 1000)
    b.pop()

  evals = np.array(evals)
  exp = np.exp(evals - np.max(evals))
  probs = exp / np.sum(exp)
  legal_moves = [str(m) for m in b.legal_moves]
  return sorted(list(zip(legal_moves, probs)), key=lambda x: x[1], reverse=True)


def maia_probs(fen):
  state = GameState(fen)
  output = maia.evaluate(state.as_input(maia))[0]
  return sorted(list(zip(state.moves(), output.p_softmax(*state.policy_indices()))), key=lambda x: x[1], reverse=True)

In [12]:
fen = '1Bb3BN/R2Pk2r/1Q5B/4q2R/2bN4/4Q1BK/1p6/1bq1R1rb w - - 0 1'

# stockfish
print('Stockfish')
board = chess.Board(fen)
for i in range(5):
  probs = stockfish_probs(board.fen())
  move = probs[0][0]
  print(i, board.fen(), board.san(chess.Move.from_uci(move)))
  board.push(chess.Move.from_uci(move))
  # print(board, end='\n\n')
  if board.is_game_over():
    print(board.result())
    break

# maia
print('Maia')
board = chess.Board(fen)
for i in range(5):
  probs = maia_probs(board.fen())
  move = probs[0][0]
  # print algebraic notation eg: Nf3
  print(i, board.fen(), board.san(chess.Move.from_uci(move)))
  board.push(chess.Move.from_uci(move))
  # print(board, end='\n\n')
  if board.is_game_over():
    print(board.result())
    break

Stockfish
0 1Bb3BN/R2Pk2r/1Q5B/4q2R/2bN4/4Q1BK/1p6/1bq1R1rb w - - 0 1 Qa3#
1-0
Maia
0 1Bb3BN/R2Pk2r/1Q5B/4q2R/2bN4/4Q1BK/1p6/1bq1R1rb w - - 0 1 Qxe5+
1 1Bb3BN/R2Pk2r/1Q5B/4Q2R/2bN4/6BK/1p6/1bq1R1rb b - - 0 1 Be6+
2 1Bb3BN/R2Pk2r/1Q2b2B/4Q2R/3N4/6BK/1p6/1bq1R1rb w - - 1 2 Nf5+
3 1Bb3BN/R2Pk2r/1Q2b2B/4QN1R/8/6BK/1p6/1bq1R1rb b - - 2 2 Bxf5+
4 1Bb3BN/R2Pk2r/1Q2b2B/4Qb1R/8/6BK/1p6/2q1R1rb w - - 0 3 Kh4
